In [12]:
import pyspark
import pandas as pd 
from pyspark.sql import SparkSession

In [19]:
os.getcwd()

'/home/ferman/Documents/FERMAN/pyspark'

In [13]:
#Define the sparksession
spark = SparkSession.builder.appName("pysparkBasis").getOrCreate()

In [79]:
# import pfile 
df = spark.read.option("header","true").csv("/home/ferman/Documents/data/NameAge.csv")
df.show()

+------+---+
|  name|age|
+------+---+
| prash| 35|
|Serhat| 28|
| roxto| 45|
| Malim| 25|
+------+---+



In [80]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)



In [81]:
df.select("name").show()

+------+
|  name|
+------+
| prash|
|Serhat|
| roxto|
| Malim|
+------+



In [82]:
df.select(["name","age"]).show()

+------+---+
|  name|age|
+------+---+
| prash| 35|
|Serhat| 28|
| roxto| 45|
| Malim| 25|
+------+---+



In [83]:
type(df.select(["name","age"]))

pyspark.sql.dataframe.DataFrame

In [84]:
df.dtypes

[('name', 'string'), ('age', 'string')]

In [85]:
#check if we have a dataframe and format of columns
df.describe()

DataFrame[summary: string, name: string, age: string]

In [86]:
df.describe().show()

+-------+-----+----------------+
|summary| name|             age|
+-------+-----+----------------+
|  count|    4|               4|
|   mean| null|           33.25|
| stddev| null|8.88350531415762|
|    min|Malim|              25|
|    max|roxto|              45|
+-------+-----+----------------+



In [89]:
df = df.withColumn("add5years",df['age']+2)
df.show()


+------+---+---------+
|  name|age|add5years|
+------+---+---------+
| prash| 35|     37.0|
|Serhat| 28|     30.0|
| roxto| 45|     47.0|
| Malim| 25|     27.0|
+------+---+---------+



In [91]:
# drop columns
df = df.drop("add5years")
df.show()


+------+---+
|  name|age|
+------+---+
| prash| 35|
|Serhat| 28|
| roxto| 45|
| Malim| 25|
+------+---+



In [93]:
#Rename columns
df = df.withColumnRenamed("name","surname")
df.show()

+-------+---+
|surname|age|
+-------+---+
|  prash| 35|
| Serhat| 28|
|  roxto| 45|
|  Malim| 25|
+-------+---+



# MISSING VALUES 